In [151]:
from os import listdir
from os.path import join
from scipy import io
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import pickle
import torch
import torch.nn as nn

In [152]:
root_dir_zero = "/content/zero/0"
zero = listdir(root_dir_zero)

root_dir_one = "/content/one/1"
one = listdir(root_dir_one)

root_dir_two = "/content/two/2"
two = listdir(root_dir_two)

root_dir_three = "/content/three/3"
three = listdir(root_dir_three)

In [153]:
def data_to_dict(file_name, root_dir, condition):
    dataframes = []
    for sub_dir in file_name:
        if 'readme' not in sub_dir:
            mat = io.loadmat(join(root_dir, sub_dir))
            features = ['AE', 'disp', 'power', 'sound']
            feat_dict = {}
            for i, feat in enumerate(features):
                feat_dict[feat] = mat["processed"][0][0][i].flatten()
            dataframes.append(pd.DataFrame(feat_dict, index=np.arange(len(mat["processed"][0][0][0]))))
    return dataframes

In [154]:
all_files = []
zero_dict = data_to_dict(zero, root_dir_zero, 0)
one_dict = data_to_dict(one, root_dir_one, 1)
two_dict = data_to_dict(two, root_dir_two, 2)
three_dict = data_to_dict(three, root_dir_three, 3)
all_files.append(zero_dict)
all_files.append(one_dict)
all_files.append(two_dict)
all_files.append(three_dict)

In [155]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [156]:
data = []
for i, f in enumerate(all_files):
    for df in f:
        #if len(df) == 100001:    
        df = df.iloc[::100, :]
        df = df.iloc[:1000, :]
        data.append([df, i])

In [157]:
class TSdata(Dataset):
    def __init__(self, data):
        self.x = []
        self.y = []
        self.data = data
        
    def __getitem__(self,index):
        df = self.data[index][0]
        self.x = torch.tensor(df[["AE", "disp", "power", "sound"]].values,dtype = torch.double)
        self.y = torch.tensor(self.data[index][1], dtype = torch.long)
        return [self.x,self.y]
    
    def __len__(self):
        return len(self.data)

In [168]:
batch_size = 10
input_size = 4
seq_len = 1000
num_layers = 4
hidden_size = 32
num_classes = 4
learning_rate = 2e-3
num_epochs = 100
dataset = TSdata(data)
loader = DataLoader(dataset, batch_size)

In [169]:
train_data, test_data = random_split(dataset, [170,30])
    

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)

Num Train: 170
Num Test: 30


In [170]:
class _RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, device):
        super(_RNN,self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True).to(device)
        #x -> (batch_size, seq_len, input_size)
        self.fc = nn.Linear(hidden_size, num_classes).to(device)
    
    def forward(self,x):
        
        x = x.double()
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device).double()
        
        out, _ = self.rnn(x, h0)
        
        #out ->  (batch_size, seq_len, hidden_size)
        
        out = out[: , -1, :]
        
        # out -> (batch_size, hidden_size)
        
        out = self.fc(out)
        
        return out

In [171]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [172]:
model = _RNN(input_size, hidden_size, num_layers, num_classes, device=device).double()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [173]:
#Training the RNN: 

total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (signals, label) in enumerate(train_loader):
                # forward pass
        signals, label = signals.to(device), label.to(device)
        outputs = model(signals)
        loss = criterion(outputs, label)
        
        #backward and optimization 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 0:
        model.eval()
        for i, (signals_test, label_test) in enumerate(test_loader):
            signals_test, label_test = signals_test.to(device), label_test.to(device)
            output_test = model(signals_test)
            val_loss = criterion(output_test, label_test)
        model.train()
        print(f'epoch [{epoch + 1} / {num_epochs}], Step [{i+1}/ {total_steps}], Loss: {loss.item()}, Val Loss: {val_loss.item()}')


epoch [1 / 100], Step [3/ 17], Loss: 1.3536930875177968, Val Loss: 1.4441991783597536
epoch [1 / 100], Step [3/ 17], Loss: 1.4823017318604443, Val Loss: 1.502930738567915
epoch [1 / 100], Step [3/ 17], Loss: 1.5059620879424822, Val Loss: 1.5302279790178448
epoch [1 / 100], Step [3/ 17], Loss: 1.4180838848516, Val Loss: 1.406773568291877
epoch [1 / 100], Step [3/ 17], Loss: 1.3921533918384514, Val Loss: 1.4309984922524304
epoch [1 / 100], Step [3/ 17], Loss: 1.3520453843094242, Val Loss: 1.43048505603419
epoch [1 / 100], Step [3/ 17], Loss: 1.4066829244830144, Val Loss: 1.440667702926477
epoch [1 / 100], Step [3/ 17], Loss: 1.4197520802964712, Val Loss: 1.4162296056700674
epoch [1 / 100], Step [3/ 17], Loss: 1.376600007091323, Val Loss: 1.4173240630237616
epoch [1 / 100], Step [3/ 17], Loss: 1.369653643098577, Val Loss: 1.4100882521671891
epoch [1 / 100], Step [3/ 17], Loss: 1.383489906068543, Val Loss: 1.4339631382437865
epoch [1 / 100], Step [3/ 17], Loss: 1.3868071497081471, Val Loss

KeyboardInterrupt: ignored